<a href="https://colab.research.google.com/github/KorStats/topicmodeling/blob/main/2_KoBERT_%EC%8B%A0%EB%AC%B8%EA%B8%B0%EC%82%AC_%EC%9A%94%EC%95%BD_TextRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [1]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 req

In [2]:
!pip install transformers

In [3]:

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [4]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 사용할 데이터

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   법률_train_original.csv   신문기사_train_original.csv   이전
'My Drive'	    법률_valid_original.csv   신문기사_valid_original.csv   졸업논문


In [6]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/신문기사_요약_0.2.csv', encoding='utf-8-sig')

In [7]:
df['category'].value_counts()

,count
category,
종합,4000
경제,2000
사회,2000
정치,2000
스포츠,1000
"IT,과학",500
교육/입시/NIE,500


In [8]:
df.head(2)

,Unnamed: 0.1,Unnamed: 0,id,category,media_type,media_sub_type,publish_date,title,text,abstractive,길이,summary_1
0,0,0,341432813,종합,online,지역지,2019-04-15 09:44:25,"홍성군, 여성 농업인 위한 다양한 정책","마을 공동급식, 출산농가 도우미 지원, 행복바우처 사업 등 [홍성]홍성군은 올해 마...","홍성군은 올해 농촌마을 44개소 마을 공동급식소 운영비 6600만 원, 농촌공동아이...",1086,군은 여성농업인 출산 시 농가도우미 지원을 위해 3000만 원의 사업비를 투입한다 ...
1,1,1,357866841,종합,online,경제지,2019-08-28 15:27:00,"이대훈 행장의 파격...""입주연장 원하는 기업, 더 있어라""",NH농협은행이 28일 농협중앙회 본관 대강당에서 'NH디지털 챌린지+' 1기 데모데...,이대훈 NH농협은행장은 28일 농협중앙회 본관 대강당에서 열린 'NH 디지털 챌린지...,1835,28일 농협중앙회 본관 대강당에서 열린 'NH 디지털 챌린지+' 1기 데모데이 행사...


In [9]:
data_list = []
for q, label in zip(df['summary_1'], df['category'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(dataset_train)
test=pd.DataFrame(dataset_test)

train.rename(columns = {0 : 'summary_1', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'summary_1', 1 : 'category'}, inplace = True)

In [10]:
train.shape

(9000, 2)

In [11]:
test.shape

(3000, 2)

# 데이터 정제

In [12]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [13]:
train['summary_1'] = clean_korean_documents(train['summary_1'])
train['summary_1'] = train['summary_1'].apply(lambda x: ','.join(x.split()))
train['summary_1'] = train['summary_1'].apply(remove_words_with_numbers)
train['summary_1'] = train['summary_1'].apply(remove_non_korean)

test['summary_1'] = clean_korean_documents(test['summary_1'])
test['summary_1'] = test['summary_1'].apply(lambda x: ','.join(x.split()))
test['summary_1'] = test['summary_1'].apply(remove_words_with_numbers)
test['summary_1'] = test['summary_1'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 3000/3000 [00:00<00:00, 9997.36it/s]


In [14]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['summary_1']
y_train = train['category']
X_test = test['summary_1']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# KoBERT

In [15]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [16]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['summary_1'], padding='max_length', truncation=True, max_length=256)

In [17]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'summary_1': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'summary_1': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [18]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 최적의 하이퍼파라미터 찾기

In [19]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        per_device_train_batch_size=batch_size,
        num_train_epochs=3,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        report_to="none"
    )

    # Trainer 생성
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1"]


# Optuna 탐색
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", study.best_params)

[I 2025-01-28 07:50:24,161] A new study created in memory with name: no-name-3aa9c0d2-8aff-4bd8-921b-076e549e2d20
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,1.743300,1.748360,0.070352
2,1.756100,1.741223,0.070352
3,1.801400,1.740679,0.070352


[I 2025-01-28 08:11:45,459] Trial 0 finished with value: 0.07035175879396985 and parameters: {'learning_rate': 0.00020236559756747335, 'batch_size': 64, 'weight_decay': 0.17218433377193723}. Best is trial 0 with value: 0.07035175879396985.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to 

Epoch,Training Loss,Validation Loss,F1
1,1.727600,1.751438,0.070352
2,1.696300,1.744674,0.070352
3,1.813100,1.741268,0.070352


[I 2025-01-28 08:33:25,871] Trial 1 finished with value: 0.07035175879396985 and parameters: {'learning_rate': 0.0003124068292356648, 'batch_size': 32, 'weight_decay': 0.22799656636119808}. Best is trial 0 with value: 0.07035175879396985.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to b

...

Epoch,Training Loss,Validation Loss,F1
1,1.061100,1.111884,0.556900
2,0.891400,0.997624,0.611354
3,0.652000,0.987363,0.638132


[I 2025-01-28 09:16:26,281] Trial 3 finished with value: 0.6381322819347243 and parameters: {'learning_rate': 5.422400782607161e-05, 'batch_size': 32, 'weight_decay': 0.2157120368650379}. Best is trial 3 with value: 0.6381322819347243.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be a

Epoch,Training Loss,Validation Loss,F1
1,1.052800,1.066076,0.583671
2,0.885200,0.969273,0.622030
3,0.738800,0.953185,0.636615


[I 2025-01-28 09:37:49,223] Trial 4 finished with value: 0.6366153139973163 and parameters: {'learning_rate': 3.0795936941248117e-05, 'batch_size': 64, 'weight_decay': 0.22711430198838575}. Best is trial 3 with value: 0.6381322819347243.


Best Hyperparameters: {'learning_rate': 5.422400782607161e-05, 'batch_size': 32, 'weight_decay': 0.2157120368650379}


Best Hyperparameters: {'learning_rate': 5.422400782607161e-05, 'batch_size': 32, 'weight_decay': 0.2157120368650379}

# 최적의 하이퍼파라미터로 재학습

In [20]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [21]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Best Hyperparameters: {'learning_rate': 5.422400782607161e-05, 'batch_size': 32, 'weight_decay': 0.2157120368650379}

In [22]:
# 7. 최적의 파라미터로 학습 설정
# 최적의 하이퍼파라미터를 찾은 후, training_args를 설정합니다.
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=3,  # 에포크 수 고정
    per_device_train_batch_size=32,  # 배치 크기 고정
    learning_rate=5.422400782607161e-05,  # 최적화된 학습률
    weight_decay=0.2157120368650379,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,1.038300,1.070823,0.575853
2,0.872900,0.977338,0.623946
3,0.693800,0.958393,0.659052


Elapsed Time: 365.00 seconds
Average CPU Usage: 64.75%


In [ ]:
# 모델 저장
joblib.dump(trainer7, '/content/gdrive/MyDrive/trainer_news_text.pkl')

print("모델이 저장되었습니다.")

모델이 저장되었습니다.


In [ ]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

loaded_model = joblib.load('/content/gdrive/MyDrive/trainer7.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [ ]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,국세청 적발 탈세 등 악용 우려 세종충청일보 장중식 기자 신용카드 위장가맹점 적발 ...,사회
1,한전 정부혁신박람회 이용환 기자 한전이 미세먼지를 청소하는 수소버스로 제 대한민국 ...,경제
2,이데일리 한광범 기자 브로드밴드는 텔레콤 와 함께 랜케이블망에서도 사용자에게 상하...,"IT,과학"
3,조배숙 의원 예산정책협 촉구 착공 뒤 착공 김형민 기자 전북 국회의원전라북도 예산정...,정치
4,교육청과 학교가 나서 현실적 대책 마련해야 홍성장 기자 광주지역 초등학교병설유치원에...,교육/입시/NIE
5,카드사의 수수료율 인상 방침에 대형 유통업계가 마케팅 축소를 검토하고 있다 이데일리...,경제
6,태광그룹 계열 한국케이블텔레콤 대표 신동선이 우체국에서 알뜰폰 판매를 시작했다 의 ...,경제
7,박길수 기자 폭염 속 농업인에게 큰 도움 오는 수확작업 대행 핵심공약 축산농가 경쟁...,종합
8,이홍재 기자 한강유역환경청은 환경부가 전국 지자체를 대상으로 실시한 공공하수도의 운...,종합
9,삼성전자가 개척한 글로벌 폴더블폰 시장에 화웨이모토로라가 가세하면서 한중 맞대결이 ...,"IT,과학"


In [ ]:
selected_columns7